# This file is temporary (will create python file for visualization and outcomes)

In this notebook we will take a dive into the costs and profits of all our orders. We will see, which products are the most profitable and which ones should, perhaps, be taken of the list.   
Among others, we will see:
- which product categories are ordered most often;
- which product categories are most often cancelled or possibly fraudulant;
- which products in each category bring in the most and the least profit;
- how do the customers types differ in the things they order;
- on which categories and which products is the company losing the most;
- what are the most and least expensive products;
- which categories are the most and least expensive, and averages, distribution of prices, kurtosis;
- which categories are the most discounted ones;
- how many sales are made per customer and sales total per customer;
- do small customer clustering via their buying patterns and PCA;
- if there are any trends in the order time and dates;
- if there is any weekly pattern in the shipement;
- many others ;)

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm
from scipy.stats import kurtosis, skew
from datetime import datetime
import pickle

pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:.2f}'.format

with open('data/supply_chain_dictionaties.pkl', 'rb') as file:
    dictionaries = pickle.load(file)

product_name = dictionaries[0]
department_name = dictionaries[1]
category_name = dictionaries[2]

In [7]:
df = pd.read_parquet('data/SupplyChainDataset_cleaned.parquet')
df.head()


,Type,Benefit per order,Sales per customer,Late_delivery_risk,Category Id,Customer City,Customer Country,Customer Id,Customer Segment,Customer State,Department Id,Market,Order Country,Order date,Order Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order Status,Product Card Id,Product Price,Shipping date,Shipping Mode,Order time,Order day,Shipping time,Shipping day,Target shipping days
0,DEBIT,91.25,314.64,0,73,Caguas,Puerto Rico,20755,Consumer,PR,2,Pacific Asia,Indonesia,2018-01-31 22:56:00,77202,13.11,0.04,180517,327.75,0.29,1,327.75,314.64,91.25,Southeast Asia,COMPLETE,1360,327.75,2018-03-02 22:56:00,Standard Class,2018-01-31 22:56:00,Wednesday,2018-03-02 22:56:00,Friday,-1
1,TRANSFER,-249.09,311.36,1,73,Caguas,Puerto Rico,19492,Consumer,PR,2,Pacific Asia,India,2018-01-13 12:27:00,75939,16.39,0.05,179254,327.75,-0.80,1,327.75,311.36,-249.09,South Asia,PENDING,1360,327.75,2018-01-18 12:27:00,Standard Class,2018-01-13 12:27:00,Saturday,2018-01-18 12:27:00,Thursday,1
2,CASH,-247.78,309.72,0,73,San Jose,United States,19491,Consumer,CA,2,Pacific Asia,India,2018-01-13 12:06:00,75938,18.03,0.06,179253,327.75,-0.80,1,327.75,309.72,-247.78,South Asia,CLOSED,1360,327.75,2018-01-17 12:06:00,Standard Class,2018-01-13 12:06:00,Saturday,2018-01-17 12:06:00,Wednesday,0
3,DEBIT,22.86,304.81,0,73,Los Angeles,United States,19490,Home Office,CA,2,Pacific Asia,Australia,2018-01-13 11:45:00,75937,22.94,0.07,179252,327.75,0.08,1,327.75,304.81,22.86,Oceania,COMPLETE,1360,327.75,2018-01-16 11:45:00,Standard Class,2018-01-13 11:45:00,Saturday,2018-01-16 11:45:00,Tuesday,-1
4,PAYMENT,134.21,298.25,0,73,Caguas,Puerto Rico,19489,Corporate,PR,2,Pacific Asia,Australia,2018-01-13 11:24:00,75936,29.50,0.09,179251,327.75,0.45,1,327.75,298.25,134.21,Oceania,PENDING_PAYMENT,1360,327.75,2018-01-15 11:24:00,Standard Class,2018-01-13 11:24:00,Saturday,2018-01-15 11:24:00,Monday,-2


In [8]:
df = df.sort_values(by='Order date')
df = df.reset_index(drop=True)
df['Order time'] = df['Order date'].dt.time
df['Shipping time'] = df['Shipping date'].dt.time
df['Shipping date'] = df['Shipping date'].dt.date
df['Order date'] = df['Order date'].dt.date
df.head()

,Type,Benefit per order,Sales per customer,Late_delivery_risk,Category Id,Customer City,Customer Country,Customer Id,Customer Segment,Customer State,Department Id,Market,Order Country,Order date,Order Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order Status,Product Card Id,Product Price,Shipping date,Shipping Mode,Order time,Order day,Shipping time,Shipping day,Target shipping days
0,CASH,88.79,239.98,0,43,Hickory,United States,11599,Consumer,NC,7,LATAM,México,2015-01-01,1,60.00,0.20,1,299.98,0.37,1,299.98,239.98,88.79,Central America,CLOSED,957,299.98,2015-03-01,Standard Class,00:00:00,Thursday,00:00:00,Sunday,-2
1,PAYMENT,68.25,227.50,0,24,Chicago,United States,256,Consumer,IL,5,LATAM,Colombia,2015-01-01,2,22.50,0.09,3,50.00,0.30,5,250.00,227.50,68.25,South America,PENDING_PAYMENT,502,50.00,2015-04-01,Standard Class,00:21:00,Thursday,00:21:00,Wednesday,-1
2,PAYMENT,36.47,107.89,0,18,Chicago,United States,256,Consumer,IL,4,LATAM,Colombia,2015-01-01,2,22.10,0.17,4,129.99,0.34,1,129.99,107.89,36.47,South America,PENDING_PAYMENT,403,129.99,2015-04-01,Standard Class,00:21:00,Thursday,00:21:00,Wednesday,-1
3,PAYMENT,91.18,193.99,0,48,Chicago,United States,256,Consumer,IL,7,LATAM,Colombia,2015-01-01,2,6.00,0.03,2,199.99,0.47,1,199.99,193.99,91.18,South America,PENDING_PAYMENT,1073,199.99,2015-04-01,Standard Class,00:21:00,Thursday,00:21:00,Wednesday,-1
4,CASH,4.10,40.98,1,40,San Antonio,United States,8827,Home Office,TX,6,LATAM,Colombia,2015-01-01,4,9.00,0.18,5,24.99,0.10,2,49.98,40.98,4.10,South America,CLOSED,897,24.99,2015-06-01,Standard Class,01:03:00,Thursday,01:03:00,Monday,1


In [9]:
df.describe()

,Benefit per order,Sales per customer,Late_delivery_risk,Order Item Discount,Order Item Discount Rate,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Product Price,Target shipping days
count,180519.00,180519.00,180519.00,180519.00,180519.00,180519.00,180519.00,180519.00,180519.00,180519.00,180519.00,180519.00,180519.00
mean,21.97,183.11,0.55,20.66,0.10,141.23,0.12,2.13,203.77,183.11,21.97,141.23,0.57
std,104.43,120.04,0.50,21.80,0.07,139.73,0.47,1.45,132.27,120.04,104.43,139.73,1.49
min,-4274.98,7.49,0.00,0.00,0.00,9.99,-2.75,1.00,9.99,7.49,-4274.98,9.99,-2.00
25%,7.00,104.38,0.00,5.40,0.04,50.00,0.08,1.00,119.98,104.38,7.00,50.00,0.00
50%,31.52,163.99,1.00,14.00,0.10,59.99,0.27,1.00,199.92,163.99,31.52,59.99,1.00
75%,64.80,247.40,1.00,29.99,0.16,199.99,0.36,3.00,299.95,247.40,64.80,199.99,1.00
max,911.80,1939.99,1.00,500.00,0.25,1999.99,0.50,5.00,1999.99,1939.99,911.80,1999.99,4.00


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 35 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   Type                      180519 non-null  category
 1   Benefit per order         180519 non-null  float64 
 2   Sales per customer        180519 non-null  float64 
 3   Late_delivery_risk        180519 non-null  int64   
 4   Category Id               180519 non-null  category
 5   Customer City             180519 non-null  object  
 6   Customer Country          180519 non-null  category
 7   Customer Id               180519 non-null  category
 8   Customer Segment          180519 non-null  category
 9   Customer State            180519 non-null  object  
 10  Department Id             180519 non-null  category
 11  Market                    180519 non-null  category
 12  Order Country             180519 non-null  object  
 13  Order date                180

In [11]:
df.isnull().sum().sum() # no missing values

0

In [12]:
df.duplicated().sum()

0

In [13]:
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
numeric_columns

Index(['Benefit per order', 'Sales per customer', 'Late_delivery_risk',
       'Order Item Discount', 'Order Item Discount Rate',
       'Order Item Product Price', 'Order Item Profit Ratio',
       'Order Item Quantity', 'Sales', 'Order Item Total',
       'Order Profit Per Order', 'Product Price', 'Target shipping days'],
      dtype='object')

In [20]:
def extended_statistics(dataframe, columns):
    es = pd.DataFrame({'Mean': dataframe[columns].mean(),
                       'Standard deviation': dataframe[columns].std(),
                       'Median': dataframe[columns].median(),
                       'Sum': [dataframe[c].sum() for c in columns],
                       'Skewness': [skew(dataframe[c]) for c in columns],
                       'Kurtosis': [kurtosis(dataframe[c]) for c in columns],
                       'Outliers count': [len(outliers(dataframe, c)[0]) for c in columns],
                      'Lower bound': [outliers(dataframe, c)[1] for c in columns],
                      'Upper bound': [outliers(dataframe, c)[2] for c in columns]})

    return es.T

In [18]:
def outliers(dataframe, column):
    outliers = []
    q1 = np.percentile(sorted(dataframe[column]), 25)
    q3 = np.percentile(sorted(dataframe[column]), 75)
    IQR = q3 - q1
    lwr_bound = q1 - (1.5 * IQR)
    upr_bound = q3 + (1.5 * IQR)
    for i in sorted(dataframe[column]): 
        if (i < lwr_bound or i > upr_bound):
            outliers.append(i)
    return outliers, lwr_bound, upr_bound


In [21]:
extended_statistics(df, numeric_columns)

,Benefit per order,Sales per customer,Late_delivery_risk,Order Item Discount,Order Item Discount Rate,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Product Price,Target shipping days
Mean,21.97,183.11,0.55,20.66,0.10,141.23,0.12,2.13,203.77,183.11,21.97,141.23,0.57
Standard deviation,104.43,120.04,0.50,21.80,0.07,139.73,0.47,1.45,132.27,120.04,104.43,139.73,1.49
Median,31.52,163.99,1.00,14.00,0.10,59.99,0.27,1.00,199.92,163.99,31.52,59.99,1.00
Sum,3966902.97,33054402.04,98977.00,3730378.40,18353.04,25495158.68,21779.01,384079.00,36784735.01,33054402.04,3966902.97,25495158.68,102139.00
Skewness,-4.74,2.89,-0.19,3.04,0.34,3.19,-2.89,0.88,2.88,2.89,-4.74,3.19,0.03
Kurtosis,71.38,23.92,-1.96,25.23,-0.90,23.31,10.16,-0.75,23.94,23.92,71.38,23.31,-0.29
Outliers count,18942.00,1943.00,0.00,7537.00,0.00,2048.00,17300.00,0.00,488.00,1943.00,18942.00,2048.00,35701.00
Lower bound,-79.70,-110.15,-1.50,-31.48,-0.14,-174.99,-0.34,-2.00,-149.98,-110.15,-79.70,-174.99,-1.50
Upper bound,151.50,461.93,2.50,66.87,0.34,424.98,0.78,6.00,569.91,461.93,151.50,424.98,2.50


**zlicz wszystkie wartości kategorii**
`df['name'].value_counts(ascending=False)`

może zbierz te, które mają mniejsze wartości niż ileś (średnia?) jako inne
`df[].apply(funkcja)`

jak znajdziesz ile zamawia się jakich produktów, wskaż top 5 i najgorsze 5 pod względem zysków i częstości wybierania
(pewnie da się to zrobić przez group by średnią albo count?)

pousuwać outliersy ale gdy zliczymy zamówienia? czy nje?

korelogram

zyski odnotowane ze sprzedaży na dzień, dla szeregu czasowego (to przy szeregach) 
lub liczba zamówień, albo oba

sprawdzić błądzenie losowe i whitenoise dla szeregu czasowego (zyski lub liczba zamówień)

czy są jakieś przesłanki dla opóźnionej wysyłki?
(klastrowanie?)

ABC analysis??co to??

In [34]:
a = df['Product Card Id'].value_counts(ascending=False)

In [35]:
b = df.groupby('Product Card Id')['Order Item Quantity'].count().sort_values(ascending=False)

In [38]:
c = np.where(a == b)
c

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117]),)

In [ ]:
def calculate_quantities(df, column):
    counts = df[column].value_counts(ascending=False)
    